In [1]:
polar_host = "mars-xianmai-pub.mysql.polardb.rds.aliyuncs.com"
polar_port = 3306
polar_user = 'risk_ro'
polar_pswd = '1LzQNi8sMv7l'
polar_db = 'mars'


In [2]:
import pandas as pd

def df_from_sql(sql):
    import pymysql
    import pandas as pd
    connect = pymysql.Connect(
        host=polar_host,
        port=polar_port,
        user=polar_user,
        passwd=polar_pswd,
        db=polar_db
    )
    cursor = connect.cursor()
    df = pd.read_sql(sql, con=connect)
    connect.close()    
    return df

In [4]:
# 虹如要的数据；
# 1. reblue全量 7月 - 全量 6月；

sql_loan = '''
select * 
from yk_reblue_sale
'''

blue = df_from_sql(sql_loan)

june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

print('7月总交易额{}'.format(july_blue.paid_amt.sum()))
print('6月总交易额{}'.format(june_blue.paid_amt.sum()))
print('7-6月 :{}'.format(july_blue.paid_amt.sum() - june_blue.paid_amt.sum()))


7月总交易额5213445.58
6月总交易额4153047.32
7-6月 :1060398.2600000002


In [8]:
# 虹如要的数据；
# 2. reblue全量 7月 客单价 总价/人数，如果非会员用dp_id代表人数；
sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
'''

blue = df_from_sql(sql_loan)
june_blue = blue[blue.sale_time.between('2020-06-01 00:00', '2020-07-01 00:00')]
july_blue = blue[blue.sale_time.between('2020-07-01 00:00', '2020-08-01 00:00')]

july_blue = july_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]
june_blue = june_blue[['user_id', 'paid_amt', 'dp_id', 'xp_id']]

june_blue = june_blue[june_blue.paid_amt>0].iloc[:,1:]
print('6月大票数：{}'.format(june_blue.dp_id.nunique()))
print('6月单均：{}'.format(june_blue.paid_amt.sum()/june_blue.dp_id.nunique()))

july_blue = july_blue[july_blue.paid_amt>0].iloc[:,1:]
print('7月大票数:{}'.format(july_blue.dp_id.nunique()))
print('7月单均:{}'.format(july_blue.paid_amt.sum()/july_blue.dp_id.nunique()))


6月大票数：3413
6月单均：1446.8952475827718
7月大票数:4076
7月单均:1477.1944553483809


In [9]:
# 虹如要的数据；
# 3. reblue全量 5.1~6.16 交易额/天数;
sql_loan = '''
select * 
from yk_reblue_sale s
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-05-01 00:00:00', '2020-06-17 00:00:00')]
print('reblue全量 5.1~6.16 交易额/天数:{}'.format(tmp_blue.paid_amt.sum()/47))

reblue全量 5.1~6.16 交易额/天数:140410.93914893616


In [10]:
# 虹如要的数据；
# 4. reblue全量 周数据[首周22-28]；每周交易额
# 5. reblue西瓜 周数据[首周22-28]；每周交易额

sql_loan = '''
select * 
from yk_reblue_sale s
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-22 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time']]
tmp_blue['date'] = tmp_blue['sale_time'].apply(lambda x:str(x)[0:10])

def weekget(string):
    if string in ['2020-06-22','2020-06-23','2020-06-24','2020-06-25','2020-06-26','2020-06-27','2020-06-28']:
        return '上线第1周'
    elif string in ['2020-06-29','2020-06-30','2020-07-01','2020-07-02','2020-07-03','2020-07-04','2020-07-05']:
        return '上线第2周'
    elif string in ['2020-07-06','2020-07-07','2020-07-08','2020-07-09','2020-07-10','2020-07-11','2020-07-12']:
        return '上线第3周'
    elif string in ['2020-07-13','2020-07-14','2020-07-15','2020-07-16','2020-07-17','2020-07-18','2020-07-19']:
        return '上线第4周'    
    elif string in ['2020-07-20','2020-07-21','2020-07-22','2020-07-23','2020-07-24','2020-07-25','2020-07-26']:
        return '上线第5周'    
    elif string in ['2020-07-27','2020-07-28','2020-07-29','2020-07-30','2020-07-31']:
        return '上线第6周'    
    else:
        print(string)
        raise Exception('not allowed')

tmp_blue['week'] = tmp_blue.date.apply(lambda x: weekget(x))
tmp_blue.pivot_table(index='week', values=['paid_amt', 'hapay_pay'], aggfunc=sum)/7

,hapay_pay,paid_amt
week,,
上线第1周,28241.157143,155995.300000
上线第2周,87218.892857,150242.352857
上线第3周,71871.778571,158158.557143
上线第4周,60075.800000,158836.867143
上线第5周,85485.885714,196675.012857
上线第6周,50458.007143,107946.944286


In [11]:
# 虹如要的数据；
# 6. reblue西瓜 新老顾客
# 西瓜新老用户定义：7月1号之后来的，之前没来过都是新用户
sql_loan = '''
select * 
from mars_financial_settle_details f
left join mars_hapay_store s on s.refid = f.storerefid
left join mars_loan l on l.refid = f.loanrefid
left join mars_user u on u.refid = l.userrefid
where f.type='交易'
'''

blue = df_from_sql(sql_loan)

blue = blue[blue.financeRefId.isin(['WFY2_Re-BLUE_default'])]
blue['flag'] = 1
user = blue.pivot_table(index='userRefId', 
                        values=['goodsOrderAmount', 'flag'],
                        aggfunc={'goodsOrderAmount':sum,
                                 'flag':sum})


old_users = blue[blue.fundTime<'2020-07-01 00:00'].userRefId
july_blue = blue[blue.fundTime>'2020-07-01 00:00']
july_blue['是老用户'] = july_blue.userRefId.isin(old_users)
# july_blue.index='userRefId', values='是老用户', aggfunc='')
h = july_blue[['userRefId', '是老用户']].drop_duplicates().set_index('userRefId')
h['是老用户'].value_counts()


/Users/cashbus/py3env/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


False    271
True      15
Name: 是老用户, dtype: int64

In [13]:
# 虹如要的数据；
# 7. reblue 门店 月份 西瓜 ---- 总量 单数 金额;
sql_loan = '''
select * 
from yk_reblue_sale s
left join yk_reblue_user u on u.user_id = s.user_id
left join yk_reblue_saler sr on sr.saler_id = s.saler_id
'''
blue = df_from_sql(sql_loan)
tmp_blue = blue[blue.sale_time.between('2020-06-01 00:00:00', '2020-08-01 00:00:00')]
tmp_blue = tmp_blue[['paid_amt', 'hapay_pay', 'dp_id', 'user_id', 'sale_time', 'store_name']]
tmp_blue['month'] = tmp_blue['sale_time'].apply(lambda x:str(x)[5:7])


def get_df_smy(df):
    '''
    # 过滤掉负交易;
    # 由于xg最低单位是 单号，而yk最低单位是goods,所以不匹配要先矫正【矫正方法有待商榷】;
    1. 销售金额
    2. 总大票数目
    3. 西瓜金额
    4. 西瓜总大票数目
    '''
    # 过滤掉负交易;
    df = df[df['paid_amt']>=0]
    xg_df = df[df['hapay_pay']>0].copy()
    # 由于xg最低单位是 单号，而yk最低单位是goods,所以不匹配要先矫正;
#     xg_df.loc[:, '西瓜支付'] = xg_df.loc[:, '销售实收金额']
    amt, num = df['paid_amt'].sum(), df['dp_id'].nunique()
    xg_df2 = xg_df.pivot_table(index='dp_id', values=['hapay_pay'], aggfunc={'hapay_pay':max})
    if not xg_df2.empty:
        xg_amt, xg_num = xg_df2['hapay_pay'].sum(), xg_df2.shape[0]
    else:
        xg_amt, xg_num = 0, 0
    dic = {}
    dic['销售金额'] = amt
    dic['销售单数'] = num 
    dic['西瓜金额'] = xg_amt
    dic['西瓜单数'] = xg_num
    return dic

# show1 = tmp_blue.pivot_table(index=['month', 'store_name'], 
#                              values=['paid_amt','dp_id'], 
#                              aggfunc={'paid_amt':sum, 'dp_id':'nunique'})

# xg = tmp_blue[tmp_blue.hapay_pay>0]
# xg = xg[['dp_id', 'hapay_pay', 'store_name', 'month']]
# xg = xg.drop_duplicates()
# show2 = xg.pivot_table(index=['month', 'store_name'],
#                        values=['hapay_pay','dp_id'], 
#                        aggfunc={'hapay_pay':sum, 'dp_id':'nunique'})

DIC = {}
res = []
for s in tmp_blue['store_name'].unique():
    print(s)
    tmp_df = tmp_blue[tmp_blue['store_name']==s]
    D = {}
    for m in ['06', '07']:
        dic = {}
        tmp = tmp_df[tmp_df['month']==m]
        dic['month'+ m] =  get_df_smy(tmp)
        D.update(dic)
    sub = pd.DataFrame(D).T
    DIC[s]= sub
    res.append(sub)

def sum_brackets(a):
    return sum(a, [])

class1 = sum_brackets([[i]*2 for i in list(tmp_blue['store_name'].unique())])
class2 =["month06","month07"]*len(tmp_blue['store_name'].unique())
m_index2=pd.MultiIndex.from_arrays([class1,class2],names=["门店","月份"])

df2=pd.DataFrame(pd.concat(res).values, index=m_index2, columns=['销售金额','销售单数','西瓜金额','西瓜单数'])


江苏南京景枫
北京国瑞城
四川成都龙湖西宸天街
重庆龙湖时代天街
南京龙湾天街
上海宝杨宝龙
杭州滨江宝龙城店
江苏苏州昆山九方城
南京中海环宇城
兰州七里河店
杭州乐堤港
江苏苏州工业园
深圳福田卓悦汇
四川成都珠江广场
河南郑州永威木色
杭州滨江龙湖天街
贵阳金阳南路店
河南郑州公园茂店
南京华采天地
北京九龙山合生汇
四川成都银泰
杭州紫荆龙湖
北京朝阳金地中心店
济南恒隆
北京大兴荟聚中心店
江苏南京印象汇
西安大融城
廊坊新朝阳
常州环球港
西安曲江新区金地
云南南亚风情城
杭州西溪印象城
呼和浩特振华店
石家庄万达
河北保定竞秀区保定购物商场
北京大兴大族广场
浙江宁波万象汇
杭州简易之间


In [14]:
df2.to_csv('/Users/cashbus/Desktop/f2.csv')